## Convolutional Networks

We'll check out how to build a **convolutional network** to classify CIFAR10 images. By using weight sharing - multiple units with the same weights - convolutional layers are able to learn repeated patterns in your data. For example, a unit could learn the pattern for an eye, or a face, or lower level features like edges.


In [1]:
import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from data_utils import load_CIFAR10

import helper

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = './data/cifar-10-batches-py'
    X_train, y_train = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    
    return X_train, y_train, X_val, y_val


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

X_train, y_train = torch.from_numpy(X_train).type(torch.FloatTensor), torch.from_numpy(y_train).type(torch.LongTensor)
X_val, y_val = torch.from_numpy(X_val).type(torch.FloatTensor), torch.from_numpy(y_val).type(torch.LongTensor)

traindataset = utils.TensorDataset(X_train, y_train)
trainloader = utils.DataLoader(traindataset, batch_size=64, shuffle=True)

valdataset = utils.TensorDataset(X_val, y_val)
valloader = utils.DataLoader(valdataset, batch_size=64, shuffle=True)

Files already downloaded and verified
Train data shape:  (49000, 3, 32, 32)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3, 32, 32)
Validation labels shape:  (1000,)


In [4]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=10):
        super().__init__()
        '''
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=2)
        self.fc1 = torch.nn.Linear(9 * 64, 200)
        self.fc2 = torch.nn.Linear(200, 10)
        '''
        self.conv32 = torch.nn.Conv2d(n_input_channels, 32, kernel_size=3, padding=1)
        self.conv64 = torch.nn.Conv2d(32, 64,kernel_size=3, padding=1)
        self.conv128_1 = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv128_2 = torch.nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv256_1 = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv256_2 = torch.nn.Conv2d(256, 256, kernel_size=5, padding=1)
        
        self.bn32 = torch.nn.BatchNorm2d(32)
        self.bn64 = torch.nn.BatchNorm2d(64)
        self.bn128 = torch.nn.BatchNorm2d(128)
        self.bn256 = torch.nn.BatchNorm2d(256)
        

        
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.pool_2 = torch.nn.AvgPool2d(1, 1)
        self.fc1 = torch.nn.Linear(256, 64)
        self.fc2 = torch.nn.Linear(64, 10)
        self.fc = torch.nn.Linear(256, n_output)

    def forward(self, x):
        '''
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 3* 32*32)))
        x = self.fc2(x)
        '''
        #x = x.view(x.size(0), -1)
        
        x = self.conv32(x)
        x = F.relu(self.bn32(x))
        x = self.pool(x)
        
        x = self.conv64(x)
        x = F.relu(self.bn64(x)) 
        x = self.pool(x)
        
        x = self.conv128_1(x)
        x = F.relu(self.bn128(x))
        x = self.conv128_2(x)
        x = F.relu(self.bn128(x))      
        x = self.pool(x)
        
        x = self.conv256_1(x)
        x = F.relu(self.bn256(x))
        x = self.conv256_2(x)
        x = F.relu(self.bn256(x))      
        x = self.pool(x)
        
        x = self.pool_2(x)
        #print(x.shape)
        x = x.view(-1, 256) 
        #print(x.shape)
        x = self.fc1(x)
        x = self.fc2(x)
        #x = self.fc(x)

        #x = x.view(-1, 16 * 5 * 5)

        return x
 
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [7]:
net = ConvNet()
################################################################################
# TODO:                                                                        #
# Choose an Optimizer that will be used to minimize the loss function.         #
# Choose a critera that measures the loss                                      #
################################################################################
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01,momentum=0.9)
#loss_func = torch.nn.CrossEntropyLoss()

epochs = 15
steps = 0
running_loss = 0
print_every = 20
for e in range(epochs):
    start = time.time()
    for images, labels in iter(trainloader):
        
        steps += 1
        ################################################################################
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        #                                                                              #
        ################################################################################
        #pass
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        
        ################################################################################|
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        # HINT: Calculate the gradient and move one step further                       #
        ################################################################################
        #pass
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        optimizer.zero_grad()
        
        output = net.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()        
        #loss = criterion(output, labels)
        running_loss += loss.item()
        
        if steps % print_every == 0:
            stop = time.time()
            # Test accuracy
            #net.eval()
            #total = 0
            accuracy = 0
            for ii, (images, labels) in enumerate(valloader):
                #outputs = net.predict(images)
                #predicted = net.predict(outputs)
                #_, predicted = torch.max(outputs.data, 1)
                outputs=net.forward(images)
                _, predicted = torch.max(F.softmax(outputs).data, 1)
                total = labels.size(0)
                accuracy += ((predicted == labels).sum().item())/total
                ################################################################################
                # TODO:                                                                        #
                # Calculate the accuracy                                                       #
                ################################################################################
                #pass
                ################################################################################
                #                              END OF YOUR CODE                                #
                ################################################################################
            
            print("Epoch: {}/{}..".format(e+1, epochs),
                  "Loss: {:.4f}..".format(running_loss/print_every),
                  "Test accuracy: {:.4f}..".format(accuracy/(ii+1)),
                  "{:.4f} s/batch".format((stop - start)/print_every)
                 )
            running_loss = 0
            start = time.time()

D:\python\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/15.. Loss: 2.0260.. Test accuracy: 0.3379.. 0.5578 s/batch
Epoch: 1/15.. Loss: 1.7301.. Test accuracy: 0.4445.. 0.5280 s/batch
Epoch: 1/15.. Loss: 1.5688.. Test accuracy: 0.4154.. 0.5833 s/batch
Epoch: 1/15.. Loss: 1.5003.. Test accuracy: 0.4553.. 0.5259 s/batch
Epoch: 1/15.. Loss: 1.4901.. Test accuracy: 0.4859.. 0.5301 s/batch
Epoch: 1/15.. Loss: 1.4747.. Test accuracy: 0.4996.. 0.5874 s/batch
Epoch: 1/15.. Loss: 1.4116.. Test accuracy: 0.5344.. 0.6406 s/batch
Epoch: 1/15.. Loss: 1.3835.. Test accuracy: 0.5227.. 0.6126 s/batch
Epoch: 1/15.. Loss: 1.3590.. Test accuracy: 0.5357.. 0.5994 s/batch
Epoch: 1/15.. Loss: 1.3184.. Test accuracy: 0.5484.. 0.5783 s/batch
Epoch: 1/15.. Loss: 1.2385.. Test accuracy: 0.5521.. 0.5784 s/batch
Epoch: 1/15.. Loss: 1.2994.. Test accuracy: 0.5764.. 0.5780 s/batch
Epoch: 1/15.. Loss: 1.1830.. Test accuracy: 0.5711.. 0.5898 s/batch
Epoch: 1/15.. Loss: 1.2036.. Test accuracy: 0.5611.. 0.5901 s/batch
Epoch: 1/15.. Loss: 1.1558.. Test accuracy: 0.59

Epoch: 4/15.. Loss: 0.4361.. Test accuracy: 0.7576.. 0.6095 s/batch
Epoch: 4/15.. Loss: 0.4749.. Test accuracy: 0.7736.. 0.6340 s/batch
Epoch: 4/15.. Loss: 0.5311.. Test accuracy: 0.7676.. 0.6181 s/batch
Epoch: 4/15.. Loss: 0.4967.. Test accuracy: 0.7805.. 0.6074 s/batch
Epoch: 4/15.. Loss: 0.5036.. Test accuracy: 0.7625.. 0.6341 s/batch
Epoch: 4/15.. Loss: 0.4922.. Test accuracy: 0.7697.. 0.6410 s/batch
Epoch: 4/15.. Loss: 0.4755.. Test accuracy: 0.7645.. 0.6033 s/batch
Epoch: 4/15.. Loss: 0.4757.. Test accuracy: 0.7637.. 0.5365 s/batch
Epoch: 4/15.. Loss: 0.4693.. Test accuracy: 0.7703.. 0.5380 s/batch
Epoch: 4/15.. Loss: 0.5137.. Test accuracy: 0.7682.. 0.5395 s/batch
Epoch: 4/15.. Loss: 0.5361.. Test accuracy: 0.7797.. 0.5470 s/batch
Epoch: 4/15.. Loss: 0.4940.. Test accuracy: 0.7752.. 0.5905 s/batch
Epoch: 4/15.. Loss: 0.5233.. Test accuracy: 0.7600.. 0.5747 s/batch
Epoch: 4/15.. Loss: 0.4456.. Test accuracy: 0.7830.. 0.5947 s/batch
Epoch: 4/15.. Loss: 0.4693.. Test accuracy: 0.75

Epoch: 7/15.. Loss: 0.2182.. Test accuracy: 0.7957.. 0.5852 s/batch
Epoch: 7/15.. Loss: 0.1972.. Test accuracy: 0.8055.. 0.5619 s/batch
Epoch: 7/15.. Loss: 0.1928.. Test accuracy: 0.7910.. 0.5535 s/batch
Epoch: 7/15.. Loss: 0.2128.. Test accuracy: 0.7898.. 0.5383 s/batch
Epoch: 7/15.. Loss: 0.2060.. Test accuracy: 0.7926.. 0.5691 s/batch
Epoch: 7/15.. Loss: 0.2088.. Test accuracy: 0.7988.. 0.5853 s/batch
Epoch: 7/15.. Loss: 0.2284.. Test accuracy: 0.7902.. 0.5798 s/batch
Epoch: 7/15.. Loss: 0.2667.. Test accuracy: 0.7975.. 0.6186 s/batch
Epoch: 7/15.. Loss: 0.2199.. Test accuracy: 0.7842.. 0.5599 s/batch
Epoch: 7/15.. Loss: 0.2518.. Test accuracy: 0.7883.. 0.5500 s/batch
Epoch: 7/15.. Loss: 0.2591.. Test accuracy: 0.7926.. 0.5440 s/batch
Epoch: 7/15.. Loss: 0.2688.. Test accuracy: 0.7945.. 0.6083 s/batch
Epoch: 7/15.. Loss: 0.2272.. Test accuracy: 0.7965.. 0.5428 s/batch
Epoch: 7/15.. Loss: 0.2527.. Test accuracy: 0.8020.. 0.5333 s/batch
Epoch: 7/15.. Loss: 0.2838.. Test accuracy: 0.77

Epoch: 10/15.. Loss: 0.0781.. Test accuracy: 0.8008.. 0.5728 s/batch
Epoch: 10/15.. Loss: 0.0723.. Test accuracy: 0.7965.. 0.5423 s/batch
Epoch: 10/15.. Loss: 0.0861.. Test accuracy: 0.8006.. 0.5497 s/batch
Epoch: 10/15.. Loss: 0.0695.. Test accuracy: 0.8148.. 0.5837 s/batch
Epoch: 10/15.. Loss: 0.0835.. Test accuracy: 0.8047.. 0.5485 s/batch
Epoch: 10/15.. Loss: 0.0830.. Test accuracy: 0.7920.. 0.5383 s/batch
Epoch: 10/15.. Loss: 0.0895.. Test accuracy: 0.8004.. 0.5259 s/batch
Epoch: 10/15.. Loss: 0.1009.. Test accuracy: 0.8070.. 0.5205 s/batch
Epoch: 10/15.. Loss: 0.0808.. Test accuracy: 0.8064.. 0.5266 s/batch
Epoch: 10/15.. Loss: 0.0989.. Test accuracy: 0.7893.. 0.5239 s/batch
Epoch: 10/15.. Loss: 0.1062.. Test accuracy: 0.7975.. 0.5279 s/batch
Epoch: 10/15.. Loss: 0.1117.. Test accuracy: 0.7875.. 0.5255 s/batch
Epoch: 10/15.. Loss: 0.0941.. Test accuracy: 0.8021.. 0.5692 s/batch
Epoch: 10/15.. Loss: 0.1340.. Test accuracy: 0.7957.. 0.5248 s/batch
Epoch: 10/15.. Loss: 0.1310.. Test

Epoch: 13/15.. Loss: 0.0537.. Test accuracy: 0.8127.. 0.5764 s/batch
Epoch: 13/15.. Loss: 0.0560.. Test accuracy: 0.8154.. 0.5544 s/batch
Epoch: 13/15.. Loss: 0.0453.. Test accuracy: 0.7988.. 0.5532 s/batch
Epoch: 13/15.. Loss: 0.0637.. Test accuracy: 0.8061.. 0.5316 s/batch
Epoch: 13/15.. Loss: 0.0535.. Test accuracy: 0.8047.. 0.5353 s/batch
Epoch: 13/15.. Loss: 0.0545.. Test accuracy: 0.8184.. 0.5408 s/batch
Epoch: 13/15.. Loss: 0.0661.. Test accuracy: 0.7994.. 0.5267 s/batch
Epoch: 13/15.. Loss: 0.0786.. Test accuracy: 0.8104.. 0.5379 s/batch
Epoch: 13/15.. Loss: 0.0611.. Test accuracy: 0.7971.. 0.5443 s/batch
Epoch: 13/15.. Loss: 0.0618.. Test accuracy: 0.8117.. 0.5484 s/batch
Epoch: 13/15.. Loss: 0.0459.. Test accuracy: 0.7967.. 0.5366 s/batch
Epoch: 13/15.. Loss: 0.0625.. Test accuracy: 0.7994.. 0.5313 s/batch
Epoch: 13/15.. Loss: 0.0597.. Test accuracy: 0.7957.. 0.5396 s/batch
Epoch: 13/15.. Loss: 0.0518.. Test accuracy: 0.7959.. 0.5377 s/batch
Epoch: 13/15.. Loss: 0.0786.. Test

Save best trained model.

In [9]:
torch.save(net.state_dict(), 'model_vgg_32.ckpt')